# Setup

In [32]:
import json
import os
import sys
from itertools import chain, combinations

sys.path.append('..')
sys.path.append('../scripts')

from tqdm import tqdm

import pandas as pd

from utils import read_csv_non_utf
from train_final_model import main as train_candidate_model

In [2]:
# Loading in general configuration
with open('../config.json', 'r') as f:
    config = json.load(f)

# Getting filepaths
gdrive_fp = config['gdrive_path']
LIFE_fp = config['LIFE_folder']
dataset_fp = config['datasets_path']
benitez_lopez2019 = config['indiv_data_paths']['benitez_lopez2019_recreated']

#  reading the dataset
ben_lop_path = os.path.join(gdrive_fp, LIFE_fp, dataset_fp, benitez_lopez2019)
data = read_csv_non_utf(ben_lop_path)

data.head()

,Reference,Study,Order,Family,Species,Species_List,Longitude,Latitude,Response_Ratio,Region,...,Road_Density,Percent_Settlement_50km,BM,DistKm,Reserve,TravTime,LivestockBio,Stunting,PopDens,Literacy
0,"Laurance et al., 2006",1,Cetartiodactyla,Bovidae,"Cephalophus callipygus, C. dorsalis, C. leucog...",Cephalophus callipygus|Cephalophus dorsalis|Ce...,9.839,-1.916,0.377193,Africa,...,129,0.002549,17.07,0.05,No,755.8,39.25948,22.0,0.86,81.8
1,"Laurance et al., 2006",1,Proboscidea,Elephantidae,Loxodonta africana,Loxodonta africana,9.839,-1.916,0.865690,Africa,...,129,0.002549,3940.03,0.05,No,755.8,39.25948,22.0,0.86,81.8
2,"Laurance et al., 2006",1,Cetartiodactyla,Bovidae,"Cephalophus callipygus, C. dorsalis, C. leucog...",Cephalophus callipygus|Cephalophus dorsalis|Ce...,9.839,-1.916,0.833333,Africa,...,129,0.002549,17.07,0.30,No,755.8,39.25948,22.0,0.86,81.8
3,"Laurance et al., 2006",1,Proboscidea,Elephantidae,Loxodonta africana,Loxodonta africana,9.839,-1.916,0.900862,Africa,...,129,0.002549,3940.03,0.30,No,755.8,39.25948,22.0,0.86,81.8
4,"Laurance et al., 2006",1,Cetartiodactyla,Bovidae,"Cephalophus callipygus, C. dorsalis, C. leucog...",Cephalophus callipygus|Cephalophus dorsalis|Ce...,9.839,-1.916,0.956140,Africa,...,129,0.002549,17.07,0.60,No,755.8,39.25948,22.0,0.86,81.8


## Utility functions

In [3]:
# Powerset function, ignoring the case of empty subset 
#  - https://docs.python.org/3/library/itertools.html#itertools-recipes  
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1))

# Getting combos of features to try

In [4]:
# Defining the number of models to compare for model selection
predictors = ['Body_Mass', 'Stunting_Pct', 'Literacy_Rate', 'Dist_Settlement_KM', 'Travel_Time_Large',
              'Protected_Area', 'Livestock_Biomass', 'Population_Density', 'Percent_Settlement_50km']
interactions = ['Body_Mass*Dist_Settlement_KM', 'Body_Mass*Percent_Settlement_50km']
quadratics = ['I(Dist_Settlement_KM^2)', 'I(Population_Density^2)', 'I(Body_Mass^2)', 
              'I(Stunting_Pct^2)']

all_predictors = predictors + interactions + quadratics
all_combos = list(powerset(all_predictors))

In [5]:
# Thinning out possible combinations w/some reasonable rules:
#  1. Only include a quadratic if the original var is included
#  2. Only include interaction terms if both vars are included
#  3. Models must include body mass, population density, and at least one of 
#     the distance to settlement measures
final_combos = []

for c in all_combos:
    include = True
    
    #  rule 1
    for q in quadratics:
        if q in c:
            original_var = q.removeprefix('I(').removesuffix('^2)')
            
            if original_var not in c:
                include = False

    #  rule 2
    for i in interactions:
        if i in c:
            var1 = i.split('*')[0]
            var2 = i.split('*')[1]

            if (var1 not in c) or (var2 not in c):
                include = False

    #  rule 3
    if 'Body_Mass' not in c:
        include = False
    elif 'Population_Density' not in c:
        include = False
    elif ('Dist_Settlement_KM' not in c) or ('Percent_Settlement_50km' not in c):
        include = False

    #  including only if it satisfies all three rules
    if include:
        final_combos.append(c)

In [6]:
# Checking out the resulting set of models to test
print(f'After thinning out based on the ruleset, there are {len(final_combos)} models to try')
print(f'All models are unique: {len(set(final_combos)) == len(final_combos)}')

After thinning out based on the ruleset, there are 1536 models to try
All models are unique: True


In [13]:
# Constructing the formula based on the var subsets to try
nonzero_formula = []
zero_formula = []

for c in final_combos:
    formula = ' + '.join(c)
    nonzero_formula.append('RR ~ ' + formula + ' + (1|Country) + (1|Species) + (1|Study)')
    zero_formula.append('local_extirpation ~ ' + formula + ' + (1|Country) + (1|Species) + (1|Study)')

# Running the models + getting BIC

In [14]:
# A wrapper class to pass in params to the script like argparse
class ModelParams:
    def __init__(self, params):
        for k, v in params.items():
            setattr(self, k, v)

In [15]:
# Establishing generic model params
params = {'gdrive' : True,
          'dataset' : 'mammals_recreated', 
          'rebalance_dataset' : False,
          'tune_thresh' : False, 
          'outlier_cutoff' : 5, 
          'model_to_use' : 'pymer',
          'return_model' : True,
          'save_trained_model' : False,
          'verbose' : False}

In [52]:
# Testing each candidate formula for the zero/nonzero models + recording the results...
formula = list(zip(zero_formula, nonzero_formula))[ : 100]
zero_results = {'formula' : [], 'aic' : [], 'bic' : [], 'log_likelihood' : []}
nonzero_results = {'formula' : [], 'aic' : [], 'bic' : [], 'log_likelihood' : []}

for z_form, nz_form in tqdm(formula):
    #  filling out the params w/the zero + nonzero formula to try
    params['pymer_zero_formula'] = z_form
    params['pymer_nonzero_formula'] = nz_form
    args = ModelParams(params)

    #  train the zero + nonzero models
    trained_model = train_candidate_model(args)

    #  record results in terms of AIC, BIC, and log-likelihood
    zero_results['formula'].append(z_form)
    zero_results['aic'].append(trained_model.zero_model.model.AIC)
    zero_results['bic'].append(trained_model.zero_model.model.BIC)
    zero_results['log_likelihood'].append(trained_model.zero_model.model.logLike)

    nonzero_results['formula'].append(nz_form)
    nonzero_results['aic'].append(trained_model.nonzero_model.model.AIC)
    nonzero_results['bic'].append(trained_model.nonzero_model.model.BIC)
    nonzero_results['log_likelihood'].append(trained_model.nonzero_model.model.logLike)

 10%|█████▉                                                     | 10/100 [00:13<02:11,  1.46s/it]

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 1 negative eigenvalues 



 19%|███████████▏                                               | 19/100 [00:23<01:26,  1.07s/it]

[1] "Model failed to converge with max|grad| = 0.122257 (tol = 0.002, component 1)"
[2] " \n"                                                                          

[1] "Model is nearly unidentifiable: very large eigenvalue\n - Rescale variables?"
[2] " \n"                                                                         



 40%|███████████████████████▌                                   | 40/100 [00:49<01:08,  1.13s/it]

...

 79%|██████████████████████████████████████████████▌            | 79/100 [01:43<00:22,  1.09s/it]

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 1 negative eigenvalues 



 97%|█████████████████████████████████████████████████████████▏ | 97/100 [02:10<00:04,  1.36s/it]

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 1 negative eigenvalues 



100%|██████████████████████████████████████████████████████████| 100/100 [02:15<00:00,  1.36s/it]


In [53]:
# Turning results into dataframes + sorting by BIC
zero_results_df = pd.DataFrame.from_dict(zero_results)
nonzero_results_df = pd.DataFrame.from_dict(nonzero_results)

zero_results_df = zero_results_df.sort_values('bic', ascending = True)
nonzero_results_df = nonzero_results_df.sort_values('bic', ascending = True)

In [58]:
# Zero results
zero_results_df.head(5)

,formula,aic,bic,log_likelihood
30,local_extirpation ~ Body_Mass + Dist_Settlemen...,2102.517851,2163.211275,-1041.258926
67,local_extirpation ~ Body_Mass + Stunting_Pct +...,2102.435639,2169.198405,-1040.217819
29,local_extirpation ~ Body_Mass + Dist_Settlemen...,2115.345480,2176.038903,-1047.672740
74,local_extirpation ~ Body_Mass + Stunting_Pct +...,2111.118101,2177.880867,-1044.559051
44,local_extirpation ~ Body_Mass + Dist_Settlemen...,2117.966985,2178.660409,-1048.983493


In [55]:
# Nonzero results
nonzero_results_df.head(5)

,formula,aic,bic,log_likelihood
0,RR ~ Body_Mass + Dist_Settlement_KM + Populati...,7033.888569,7087.283058,-3507.944284
1,RR ~ Body_Mass + Stunting_Pct + Dist_Settlemen...,7031.870902,7091.198112,-3505.935451
2,RR ~ Body_Mass + Literacy_Rate + Dist_Settleme...,7032.275667,7091.602877,-3506.137834
3,RR ~ Body_Mass + Dist_Settlement_KM + Travel_T...,7035.884444,7095.211654,-3507.942222
8,RR ~ Body_Mass + Dist_Settlement_KM + Populati...,7036.668373,7095.995584,-3508.334187
